In [21]:
import os

import numpy as np
import pandas as pd

!ls

calendar.csv	    sales_train_validation.csv	series_for_rnn.csv
create_sales.ipynb  sample_submission.csv
fig		    sell_prices.csv


In [7]:
sales = pd.read_csv('sales_train_validation.csv')
sales.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4


In [9]:
sales[sales['item_id'] == 'HOBBIES_1_001']

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
3049,HOBBIES_1_001_CA_2_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_2,CA,0,0,0,0,...,0,0,1,2,0,4,0,0,2,2
6098,HOBBIES_1_001_CA_3_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_3,CA,0,0,0,0,...,0,2,4,0,1,1,1,0,3,3
9147,HOBBIES_1_001_CA_4_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_4,CA,0,0,0,0,...,1,0,0,1,0,1,0,1,2,1
12196,HOBBIES_1_001_TX_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,TX_1,TX,0,0,0,0,...,0,0,0,0,1,0,1,0,0,2
15245,HOBBIES_1_001_TX_2_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,TX_2,TX,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
18294,HOBBIES_1_001_TX_3_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,TX_3,TX,0,0,0,0,...,0,3,0,0,0,0,0,0,0,0
21343,HOBBIES_1_001_WI_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,WI_1,WI,0,0,0,0,...,0,0,0,0,0,0,1,2,1,1
24392,HOBBIES_1_001_WI_2_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,WI_2,WI,0,0,0,0,...,1,1,0,0,0,0,0,0,0,1
27441,HOBBIES_1_001_WI_3_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,WI_3,WI,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [15]:
sales_3049 = sales.groupby('item_id')[[f'd_{i}' for i in range(1, 1914)]].sum()
(sales_3049.T).to_csv('./series_for_rnn_3049.csv', index=False, header=False)
sales_3049.head()

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
item_id,,,,,,,,,,,,,,,,,,,,,
FOODS_1_001,6,6,4,6,7,18,10,4,11,10,...,4,4,30,7,5,3,6,2,16,6
FOODS_1_002,4,5,7,4,3,4,1,7,2,4,...,5,9,4,1,3,5,5,3,3,1
FOODS_1_003,14,8,3,6,3,8,13,10,11,6,...,7,3,5,6,3,4,4,3,11,5
FOODS_1_004,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
FOODS_1_005,34,32,13,20,10,21,18,20,25,41,...,16,14,14,18,18,27,12,15,38,9


# Obtaining predictions

In [113]:
!ls

calendar.csv	    sales_train_validation.csv	series_for_rnn.csv
create_sales.ipynb  sample_submission.csv
fig		    sell_prices.csv


In [124]:
path_to_pred = './fig/src/Results'
predictions_cat = pd.DataFrame()

for filename in os.listdir(path_to_pred):
    if filename[:11] == 'TestPredict':
        test_num = filename[len('TestPredict_N'):-4]
        test_data = pd.read_csv(path_to_pred + '/' + filename, header=None)
        predictions_cat[test_num] = test_data[0][:28]
        
for i in list(range(1, 29)):
    sales[f'F{i}'] = 0
        
predictions_cat.head()

,2965,714,2154,339,2889,759,2128,2648,775,892,...,2082,2086,2125,858,330,729,2040,1114,231,642
0,2.156442,5.289262,5.652957,7.515735,1.493588,4.570102,6.388533,7.892635,7.514767,5.652447,...,33.189636,8.668409,1.493588,3.858782,12.695252,1.493588,38.891212,11.868338,31.698195,1.493588
1,2.157888,7.134268,3.857181,5.279557,1.499059,3.171864,6.757303,6.017883,6.386237,5.288562,...,36.236118,7.131063,1.499059,3.519806,11.439784,1.499059,47.148914,10.232012,43.070751,1.499059
2,2.497943,8.658546,2.163335,4.570130,1.502694,4.215883,4.926486,7.885647,8.267532,5.650224,...,41.419403,6.015922,1.502694,6.383548,7.127272,1.502694,50.539928,11.029726,49.881264,1.502694
3,2.833167,7.499574,3.865603,5.644602,2.163893,5.644602,8.632536,9.037002,10.621606,7.500838,...,53.230446,4.921099,1.504415,6.748313,6.749673,1.504415,46.887333,17.822311,58.693695,1.504415
4,2.832632,7.115410,4.209800,7.863875,2.160281,5.277504,9.799677,7.492046,18.208038,8.254022,...,44.500870,5.998373,1.504124,6.741263,10.602971,1.504124,39.242661,25.680819,45.901157,7.829211


In [138]:
sales[['F1', 'F2', 'F3']].sum(axis=0)

F1    1516.299098
F2    1527.122724
F3    1627.581295
dtype: float64







  4%|▍         | 115/3049 [00:24<03:57, 12.38it/s]

In [139]:
from tqdm import tqdm

for item, idx in tqdm(zip(sales_3049.index.array, list(range(3049))), total=3049):
    for i, value in enumerate(predictions_cat[str(idx)], 1):
        sales.loc[sales['item_id'] == item, f'F{i}'] = value / 10








  0%|          | 0/3049 [00:00<?, ?it/s]






  0%|          | 1/3049 [00:00<05:13,  9.72it/s]






  0%|          | 3/3049 [00:00<05:03, 10.04it/s]






  0%|          | 5/3049 [00:00<04:41, 10.83it/s]






  0%|          | 7/3049 [00:00<04:22, 11.61it/s]






  0%|          | 9/3049 [00:00<04:11, 12.11it/s]






  0%|          | 11/3049 [00:00<04:05, 12.40it/s]






  0%|          | 13/3049 [00:01<04:01, 12.60it/s]






  0%|          | 15/3049 [00:01<04:01, 12.57it/s]






  1%|          | 17/3049 [00:01<03:58, 12.70it/s]






  1%|          | 19/3049 [00:01<03:54, 12.90it/s]






  1%|          | 21/3049 [00:01<03:52, 13.02it/s]






  1%|          | 23/3049 [00:01<03:50, 13.12it/s]






  1%|          | 25/3049 [00:01<03:48, 13.24it/s]






  1%|          | 27/3049 [00:02<03:50, 13.09it/s]






  1%|          | 29/3049 [00:02<03:55, 12.81it/s]






  1%|          | 31/3049 [00:02<03:56, 12.74it/s]






  1%|          | 33/3049 [00:02<03:53, 12.94it/s]







  7%|▋         | 209/3049 [00:16<03:35, 13.15it/s]






  7%|▋         | 211/3049 [00:16<03:35, 13.16it/s]






  7%|▋         | 213/3049 [00:16<03:31, 13.43it/s]






  7%|▋         | 215/3049 [00:16<03:27, 13.64it/s]






  7%|▋         | 217/3049 [00:16<03:25, 13.79it/s]






  7%|▋         | 219/3049 [00:16<03:23, 13.91it/s]






  7%|▋         | 221/3049 [00:17<03:25, 13.76it/s]






  7%|▋         | 223/3049 [00:17<03:23, 13.89it/s]






  7%|▋         | 225/3049 [00:17<03:21, 14.01it/s]






  7%|▋         | 227/3049 [00:17<03:27, 13.61it/s]






  8%|▊         | 229/3049 [00:17<03:32, 13.28it/s]






  8%|▊         | 231/3049 [00:17<03:30, 13.38it/s]






  8%|▊         | 233/3049 [00:17<03:36, 13.03it/s]






  8%|▊         | 235/3049 [00:18<03:33, 13.16it/s]






  8%|▊         | 237/3049 [00:18<03:29, 13.39it/s]






  8%|▊         | 239/3049 [00:18<03:27, 13.56it/s]






  8%|▊         | 241/3049 [00:18<03:27, 13.56it/s]






  8%|▊         | 243/3049 [00:1

 14%|█▎        | 419/3049 [00:31<03:14, 13.54it/s]






 14%|█▍        | 421/3049 [00:31<03:11, 13.70it/s]






 14%|█▍        | 423/3049 [00:32<03:20, 13.09it/s]






 14%|█▍        | 425/3049 [00:32<03:29, 12.54it/s]






 14%|█▍        | 427/3049 [00:32<03:42, 11.81it/s]






 14%|█▍        | 429/3049 [00:32<03:37, 12.06it/s]






 14%|█▍        | 431/3049 [00:32<03:29, 12.50it/s]






 14%|█▍        | 433/3049 [00:32<03:25, 12.70it/s]






 14%|█▍        | 435/3049 [00:32<03:24, 12.77it/s]






 14%|█▍        | 437/3049 [00:33<03:20, 13.04it/s]






 14%|█▍        | 439/3049 [00:33<03:22, 12.88it/s]






 14%|█▍        | 441/3049 [00:33<03:18, 13.17it/s]






 15%|█▍        | 443/3049 [00:33<03:15, 13.32it/s]






 15%|█▍        | 445/3049 [00:33<03:17, 13.20it/s]






 15%|█▍        | 447/3049 [00:33<03:21, 12.91it/s]






 15%|█▍        | 449/3049 [00:34<03:24, 12.71it/s]






 15%|█▍        | 451/3049 [00:34<03:24, 12.73it/s]






 15%|█▍        | 453/3049 [00:3

 21%|██        | 629/3049 [00:47<02:57, 13.66it/s]






 21%|██        | 631/3049 [00:47<02:56, 13.67it/s]






 21%|██        | 633/3049 [00:47<02:56, 13.72it/s]






 21%|██        | 635/3049 [00:48<02:55, 13.75it/s]






 21%|██        | 637/3049 [00:48<02:57, 13.62it/s]






 21%|██        | 639/3049 [00:48<03:00, 13.37it/s]






 21%|██        | 641/3049 [00:48<03:01, 13.27it/s]






 21%|██        | 643/3049 [00:48<03:03, 13.12it/s]






 21%|██        | 645/3049 [00:48<03:03, 13.07it/s]






 21%|██        | 647/3049 [00:48<03:03, 13.10it/s]






 21%|██▏       | 649/3049 [00:49<03:06, 12.85it/s]






 21%|██▏       | 651/3049 [00:49<03:03, 13.04it/s]






 21%|██▏       | 653/3049 [00:49<03:03, 13.05it/s]






 21%|██▏       | 655/3049 [00:49<03:02, 13.11it/s]






 22%|██▏       | 657/3049 [00:49<03:00, 13.27it/s]






 22%|██▏       | 659/3049 [00:49<02:59, 13.32it/s]






 22%|██▏       | 661/3049 [00:49<02:57, 13.43it/s]






 22%|██▏       | 663/3049 [00:5

 28%|██▊       | 839/3049 [01:03<02:55, 12.58it/s]






 28%|██▊       | 841/3049 [01:03<02:51, 12.88it/s]






 28%|██▊       | 843/3049 [01:03<02:48, 13.11it/s]






 28%|██▊       | 845/3049 [01:03<02:47, 13.13it/s]






 28%|██▊       | 847/3049 [01:03<02:49, 12.96it/s]






 28%|██▊       | 849/3049 [01:04<02:52, 12.74it/s]






 28%|██▊       | 851/3049 [01:04<02:55, 12.50it/s]






 28%|██▊       | 853/3049 [01:04<02:57, 12.34it/s]






 28%|██▊       | 855/3049 [01:04<02:59, 12.22it/s]






 28%|██▊       | 857/3049 [01:04<03:00, 12.16it/s]






 28%|██▊       | 859/3049 [01:04<03:01, 12.09it/s]






 28%|██▊       | 861/3049 [01:05<02:59, 12.19it/s]






 28%|██▊       | 863/3049 [01:05<02:55, 12.48it/s]






 28%|██▊       | 865/3049 [01:05<02:48, 12.98it/s]






 28%|██▊       | 867/3049 [01:05<02:43, 13.31it/s]






 29%|██▊       | 869/3049 [01:05<02:41, 13.47it/s]






 29%|██▊       | 871/3049 [01:05<02:41, 13.47it/s]






 29%|██▊       | 873/3049 [01:0

...

 68%|██████▊   | 2077/3049 [02:35<01:13, 13.23it/s]






 68%|██████▊   | 2079/3049 [02:35<01:13, 13.28it/s]






 68%|██████▊   | 2081/3049 [02:35<01:14, 12.97it/s]






 68%|██████▊   | 2083/3049 [02:35<01:13, 13.08it/s]






 68%|██████▊   | 2085/3049 [02:36<01:12, 13.31it/s]






 68%|██████▊   | 2087/3049 [02:36<01:11, 13.47it/s]






 69%|██████▊   | 2089/3049 [02:36<01:09, 13.76it/s]






 69%|██████▊   | 2091/3049 [02:36<01:08, 13.89it/s]






 69%|██████▊   | 2093/3049 [02:36<01:09, 13.69it/s]






 69%|██████▊   | 2095/3049 [02:36<01:10, 13.61it/s]






 69%|██████▉   | 2097/3049 [02:36<01:10, 13.56it/s]






 69%|██████▉   | 2099/3049 [02:37<01:09, 13.74it/s]






 69%|██████▉   | 2101/3049 [02:37<01:08, 13.84it/s]






 69%|██████▉   | 2103/3049 [02:37<01:08, 13.87it/s]






 69%|██████▉   | 2105/3049 [02:37<01:07, 13.91it/s]






 69%|██████▉   | 2107/3049 [02:37<01:07, 13.94it/s]






 69%|██████▉   | 2109/3049 [02:37<01:07, 13.98it/s]






 69%|██████▉  

 75%|███████▍  | 2283/3049 [02:50<00:59, 12.93it/s]






 75%|███████▍  | 2285/3049 [02:50<00:58, 13.05it/s]






 75%|███████▌  | 2287/3049 [02:51<00:56, 13.37it/s]






 75%|███████▌  | 2289/3049 [02:51<00:57, 13.29it/s]






 75%|███████▌  | 2291/3049 [02:51<00:56, 13.47it/s]






 75%|███████▌  | 2293/3049 [02:51<00:55, 13.65it/s]






 75%|███████▌  | 2295/3049 [02:51<00:55, 13.52it/s]






 75%|███████▌  | 2297/3049 [02:51<00:55, 13.51it/s]






 75%|███████▌  | 2299/3049 [02:52<00:55, 13.40it/s]






 75%|███████▌  | 2301/3049 [02:52<00:58, 12.89it/s]






 76%|███████▌  | 2303/3049 [02:52<00:58, 12.70it/s]






 76%|███████▌  | 2305/3049 [02:52<00:59, 12.51it/s]






 76%|███████▌  | 2307/3049 [02:52<00:58, 12.63it/s]






 76%|███████▌  | 2309/3049 [02:52<00:56, 13.03it/s]






 76%|███████▌  | 2311/3049 [02:52<00:55, 13.23it/s]






 76%|███████▌  | 2313/3049 [02:53<00:55, 13.32it/s]






 76%|███████▌  | 2315/3049 [02:53<00:55, 13.30it/s]






 76%|███████▌ 

 82%|████████▏ | 2489/3049 [03:06<00:44, 12.57it/s]






 82%|████████▏ | 2491/3049 [03:06<00:45, 12.28it/s]






 82%|████████▏ | 2493/3049 [03:06<00:45, 12.20it/s]






 82%|████████▏ | 2495/3049 [03:07<00:45, 12.21it/s]






 82%|████████▏ | 2497/3049 [03:07<00:45, 12.26it/s]






 82%|████████▏ | 2499/3049 [03:07<00:44, 12.28it/s]






 82%|████████▏ | 2501/3049 [03:07<00:44, 12.43it/s]






 82%|████████▏ | 2503/3049 [03:07<00:43, 12.63it/s]






 82%|████████▏ | 2505/3049 [03:07<00:42, 12.88it/s]






 82%|████████▏ | 2507/3049 [03:07<00:41, 12.94it/s]






 82%|████████▏ | 2509/3049 [03:08<00:40, 13.18it/s]






 82%|████████▏ | 2511/3049 [03:08<00:41, 12.98it/s]






 82%|████████▏ | 2513/3049 [03:08<00:41, 12.86it/s]






 82%|████████▏ | 2515/3049 [03:08<00:42, 12.63it/s]






 83%|████████▎ | 2517/3049 [03:08<00:40, 13.04it/s]






 83%|████████▎ | 2519/3049 [03:08<00:40, 13.21it/s]






 83%|████████▎ | 2521/3049 [03:09<00:39, 13.27it/s]






 83%|████████▎

 88%|████████▊ | 2695/3049 [03:22<00:26, 13.56it/s]






 88%|████████▊ | 2697/3049 [03:22<00:26, 13.49it/s]






 89%|████████▊ | 2699/3049 [03:22<00:26, 13.38it/s]






 89%|████████▊ | 2701/3049 [03:22<00:25, 13.66it/s]






 89%|████████▊ | 2703/3049 [03:22<00:25, 13.76it/s]






 89%|████████▊ | 2705/3049 [03:23<00:25, 13.74it/s]






 89%|████████▉ | 2707/3049 [03:23<00:24, 13.96it/s]






 89%|████████▉ | 2709/3049 [03:23<00:24, 14.01it/s]






 89%|████████▉ | 2711/3049 [03:23<00:24, 13.76it/s]






 89%|████████▉ | 2713/3049 [03:23<00:24, 13.83it/s]






 89%|████████▉ | 2715/3049 [03:23<00:24, 13.91it/s]






 89%|████████▉ | 2717/3049 [03:23<00:24, 13.78it/s]






 89%|████████▉ | 2719/3049 [03:24<00:24, 13.35it/s]






 89%|████████▉ | 2721/3049 [03:24<00:24, 13.26it/s]






 89%|████████▉ | 2723/3049 [03:24<00:24, 13.38it/s]






 89%|████████▉ | 2725/3049 [03:24<00:23, 13.59it/s]






 89%|████████▉ | 2727/3049 [03:24<00:24, 13.18it/s]






 90%|████████▉

 95%|█████████▌| 2901/3049 [03:38<00:11, 13.16it/s]






 95%|█████████▌| 2903/3049 [03:38<00:11, 13.16it/s]






 95%|█████████▌| 2905/3049 [03:38<00:10, 13.34it/s]






 95%|█████████▌| 2907/3049 [03:38<00:10, 13.44it/s]






 95%|█████████▌| 2909/3049 [03:38<00:10, 13.48it/s]






 95%|█████████▌| 2911/3049 [03:38<00:10, 13.58it/s]






 96%|█████████▌| 2913/3049 [03:39<00:10, 13.54it/s]






 96%|█████████▌| 2915/3049 [03:39<00:09, 13.61it/s]






 96%|█████████▌| 2917/3049 [03:39<00:09, 13.55it/s]






 96%|█████████▌| 2919/3049 [03:39<00:09, 13.51it/s]






 96%|█████████▌| 2921/3049 [03:39<00:09, 13.40it/s]






 96%|█████████▌| 2923/3049 [03:39<00:09, 13.48it/s]






 96%|█████████▌| 2925/3049 [03:39<00:09, 13.41it/s]






 96%|█████████▌| 2927/3049 [03:40<00:09, 13.55it/s]






 96%|█████████▌| 2929/3049 [03:40<00:08, 13.66it/s]






 96%|█████████▌| 2931/3049 [03:40<00:08, 13.56it/s]






 96%|█████████▌| 2933/3049 [03:40<00:08, 13.21it/s]






 96%|█████████

In [149]:
# mapping = dict(zip(sales_3049.index.array, list(range(3049))))
# days = [f'F{i}' for i in range(1, 29)]

# for key, value in mapping.items():
#     sales.loc[sales['item_id'] == key, days] = predictions_cat[str(mapping[key])] / 10
    
# sales[days].head()

In [152]:
predictions = (sales[['id'] + [f'F{i}' for i in range(1, 29)]])
predictions = pd.concat([predictions, pd.read_csv('sample_submission.csv').iloc[30490:]], axis=0)
predictions.to_csv('submission.csv', index=False)